# Trabalho ENGA75

### Autor: Allan Almeida

In [ ]:
from ur5 import UR5, forward_kinematics
from math import pi
import numpy as np
import matplotlib.pyplot as plt
ur5 = UR5()
PI = pi

#### Generate Dataset

In [ ]:
import cv2
xlim = [40, 465]
ylim = [81, 395]
x_real_lim = [-1.1599511371082611, -1.759950096116547]
y_real_lim = [0.2911156981348095, -0.511156981348095]
# xlim com yreal
# ylim com xreal
x = np.random.uniform(xlim[0], xlim[1])
y = np.random.uniform(ylim[0], ylim[1])
yreal = np.interp(x, xlim, y_real_lim)
xreal = np.interp(y, ylim, x_real_lim)
ur5.bottle.getField('translation').setSFVec3f([xreal, yreal, 0.8954395296956814])
ur5.supervisor.step(ur5.timestep)
ur5.supervisor.step(ur5.timestep)
z = np.array(ur5.bottle.getOrientation()).reshape(3, 3)[-1, -1]
# se a garrafa virar, rotacionar de volta
while z < 0.97:
    ur5.bottle.getField('rotation').setSFRotation([0, 0, 1, 1.8157])
    ur5.supervisor.step(ur5.timestep)
    ur5.supervisor.step(ur5.timestep)
ur5.setup_camera()
original_img = ur5.get_image()
cp = original_img.copy()
cp = cv2.circle(cp, (int(x), int(y)), 3, (0, 0, 255), -1)
plt.imshow(cp)
plt.show()

In [ ]:
import cv2
bgr_image = cv2.cvtColor(original_img, cv2.COLOR_RGB2BGR)
# Define the lower and upper bounds for the green color range
lower_green = np.array([0, 60, 0], dtype=np.uint8)
upper_green = np.array([50, 255, 100], dtype=np.uint8)
mask = cv2.inRange(bgr_image, lower_green, upper_green)
green_image = cv2.bitwise_and(bgr_image, bgr_image, mask=mask)
green_image = cv2.cvtColor(green_image, cv2.COLOR_BGR2RGB)
final_img = cv2.cvtColor(green_image, cv2.COLOR_RGB2GRAY)
print(final_img.shape)
plt.imshow(final_img, cmap='gray')
plt.show()
# Apply thresholding to segment the bottle from the background
_, thresholded = cv2.threshold(final_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
# Threshold the image to convert it to binary (0 or 1)
binary_image = thresholded > 240
plt.imshow(binary_image, cmap='gray')
plt.show()
binary_image[0][0]

## Etapa A1

Implementar a Cinemática Direta e Inversa do Robô UR5. Validar a modelagem com o auxílio do
simulador para um conjunto de valores das variáveis das juntas (Cinemática Direta) e para um conjunto de
posições e orientações da garra (Cinemática Inversa)

#### Cinemática Direta

<img src="Cinemática Direta (DH).png" width="500">

In [ ]:
from scipy.spatial.transform import Rotation
from tabulate import tabulate
def validate_fk(ang):
    fk, _ = forward_kinematics(ang)    # Calcular cinemática direta a partir dos ângulos
    print('Cinemática Direta: ')
    print(fk)
    ur5.move_to_config(ang)     # Mover para a posição
    print('Ground Truth: ')
    gt = ur5.get_ground_truth()     # Obter Ground Truth
    print(gt)
    Rfk = fk[:3, :3]     # Obter rotação da cinemática direta
    Rgt = gt[:3, :3]     # Obter rotação do Ground Truth
    R_diff = np.dot(Rfk, Rgt.T)     # Calcular diferença entre as rotações
    r = Rotation.from_matrix(R_diff)    # Converter matriz de rotação para objeto de rotação
    axis = r.as_rotvec()    # Obter vetor de rotação
    angle = np.linalg.norm(axis)    # Obter ângulo de rotação
    Tfk = fk[:3, 3]     # Obter translação da cinemática direta
    Tgt = gt[:3, 3]     # Obter translação do Ground Truth
    dist = np.linalg.norm(Tfk - Tgt)    # Calcular distância entre as translações
    data = [['Erro de rotação', angle*180/PI, 'graus'], ['Erro de translação', dist*1000, 'mm']]    # Criar tabela
    table = tabulate(data, headers=[' ', ' ', ' '])    # Criar tabela
    print(table)    # Printar tabela

In [ ]:
validate_fk([.5*PI, .05*PI, .45*PI, 0, -.5*PI, .5*PI])

In [ ]:
validate_fk([-.5*PI, -.05*PI, -.45*PI, 0, .5*PI, .5*PI])

In [ ]:
validate_fk([PI/2, 0, 0, 0, 0, 0])

In [ ]:
validate_fk([.1*PI, .18*PI, .57*PI, 0.25*PI, 0.2, PI/2])

In [ ]:
validate_fk([-.3*PI, -.04*PI, -.8*PI, -0.17*PI, -0.5, PI/2])

In [ ]:
validate_fk([0.2, .04*PI, .8*PI, 0.17*PI, 0.3, PI/2])

#### Cinemática Inversa

<img src="gripper.jpg" width="700">

In [ ]:
ur5.move_to_pose([-0.1, -.55, .4], [PI, 0, PI/2], wrist='up')

In [ ]:
ur5.move_to_pose([-0.1, -.55, .4], [0, 0, -PI/2], wrist='down')

In [ ]:
ur5.move_to_pose([-0.1, -.55, .4], [PI/2, 0, -PI/2], wrist='down')

In [ ]:
ur5.move_to_pose([-0.1, -.45, .4], [PI/2, 0, -PI/2], wrist='down')

In [ ]:
ur5.move_to_pose([-0.1, -.45, .25], [PI/2, 0, -PI/2], wrist='down')

## Etapa A2

Implementar um sistema de planejamento de trajetórias no espaço de juntas com uma das abordagens
abaixo:

1. Trajetórias polinomiais cúbicas ou de quinto grau;
2. Linear Segments with Parabolic Blends (LSPB);
3. Minimum Time Trajectories

<b>Polinômio de Quinto Grau</b>

$q(t) = a_0+a_1 t + a_2 t^2 + a_3 t^3 + a_4 t^4 + a_5 t^5$

$\dot{q}(t) = a_1 + 2 a_2 t + 3 a_3 t^2 + 4 a_4 t^3 + 5 a_5 t^4$

$\ddot{q}(t) = 2 a_2 + 6 a_3 t + 12 a_4 t^2 + 20 a_5 t^3$

$\begin{bmatrix}
1 & t_0 & t_0^2 & t_0^3 & t_0^4 & t_0^5\\
0 & 1 & 2t_0 & 3t_0^2 & 4t_0^3 & 5t_0^4\\
0 & 0 & 2 & 6t_0 & 12t_0^2 & 20t_0^3\\
1 & t_f & t_f^2 & t_f^3 & t_f^4 & t_f^5\\
0 & 1 & 2t_f & 3t_f^2 & 4t_f^3 & 5t_f^4\\
0 & 0 & 2 & 6t_f & 12t_f^2 & 20t_f^3\\
\end{bmatrix} \cdot \begin{bmatrix}
a_0\\
a_1\\
a_2\\
a_3\\
a_4\\
a_5\\
\end{bmatrix} = \begin{bmatrix}
q_0\\
\dot{q}_0\\
\ddot{q}_0\\
q_f\\
\dot{q}_f\\
\ddot{q}_f\\
\end{bmatrix}$


$A\cdot X = B$

In [ ]:
t, max_err, mean_err, graphs, _ = ur5.move_to_config([.5*PI, .05*PI, .45*PI, 0, .5*PI, .3*PI], graph=True)

In [ ]:
titles = ['Posição', 'Velocidade', 'Aceleração', 'Solavanco']
axes = ['rad', 'rad/s', 'rad/$s^2$', 'rad/$s^3$']

for i in range(4):
    for j in range(6):
        plt.plot(graphs[4][j], graphs[i][j])
    plt.legend(['q1', 'q2', 'q3', 'q4', 'q5', 'q6'])
    plt.grid()
    plt.title(titles[i], fontsize=18)
    plt.ylabel(axes[i], rotation=0, labelpad=20)
    plt.xlabel('t [s]')
    plt.show()

## Jacobiano

In [ ]:
_, _, _, _, _ = ur5.move_to_config([0, 0, 0, 0, 0, 0])

In [ ]:
_, _, _, graphs, jacob = ur5.move_to_config([0, -PI/2, 0, 0, 0, 0], graph=True, jacob=True, duration=None)

In [ ]:
for j in range(6):
    plt.plot(graphs[4][j], graphs[1][j])
plt.legend(['q1', 'q2', 'q3', 'q4', 'q5', 'q6'])
plt.grid()
plt.title('Velocidade', fontsize=18)
plt.ylabel('rad/s', rotation=0, labelpad=20)
plt.xlabel('t [s]')
plt.show()

In [ ]:
for j in range(3):
    plt.plot(graphs[4][j], [i[j] for i in jacob])
plt.title('Jacobiano (vel. lin.)', fontsize=18)
plt.legend(['$v_x$', '$v_y$', '$v_z$'])
plt.xlabel('t [s]')
plt.ylabel('m/s', rotation=0, labelpad=20)
plt.show()

for j in range(3, 6):
    plt.plot(graphs[4][j], [i[j] for i in jacob])
plt.title('Jacobiano (vel,. ang.)', fontsize=18)
plt.legend(['$\omega_x$', '$\omega_y$', '$\omega_z$'])
plt.xlabel('t [s]')
plt.ylabel('rad/s', rotation=0, labelpad=20)
plt.show()

## Visão Computacional | Pick and Place

### Pose de Leitura

In [ ]:
ur5.move_to_pose([-0.1, -.68, .45], [PI, 0, -PI/2], wrist='up')

In [ ]:
bottle_position = ur5.get_bottle_frame()[:3, 3]

### Pre-graspping

In [ ]:
ur5.move_to_pose([bottle_position[0]+.03, bottle_position[1]+.3, bottle_position[2]+.1], [PI/2, 0, -PI/2], wrist='down')

### Graspping

In [ ]:
ur5.move_to_pose([bottle_position[0]+.03, bottle_position[1]+.02, bottle_position[2]+.1], [PI/2, 0, -PI/2], wrist='down', duration=6)

In [ ]:
ur5.actuate_gripper(1)

### Moving

In [ ]:
ur5.move_to_pose([-.7, 0, .6], [PI/2, -1.5708+.3, -PI/2], wrist='up')

In [ ]:
ur5.move_to_pose([-.64, 0.06, .6], [0, .9, 0], wrist='down')

In [ ]:
ur5.move_to_pose([-.7, 0.06, .74], [0, .0, 0], wrist='down')

### Pre-placing

In [ ]:
ur5.move_to_pose([-.4, .1, .5], [-PI/2, 0, PI/2], wrist='up')

### Placing

In [ ]:
ur5.move_to_pose([-.4, .1, .15], [-PI/2, 0, PI/2], wrist='up', duration=6)

In [ ]:
ur5.actuate_gripper(0)

In [ ]:
ur5.move_to_pose([-.4, -.1, .15], [-PI/2, 0, PI/2], wrist='up')

### Retorna pose de leitura

In [ ]:
ur5.move_to_pose([-0.1, -.55, .4], [PI, 0, -PI/2], wrist='up')